<a href="https://colab.research.google.com/github/firekuo0803/Selflearning_AI/blob/main/AI_painting_WGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###函式庫下載和匯入

In [ ]:
!pip install opendatasets --upgrade --quiet
!pip install jovian --upgrade --quiet
%matplotlib inline

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import opendatasets as od


ModuleNotFoundError: ignored

##AAAAA

In [ ]:
import os
from google.colab import drive
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as T
import torch
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import torch.nn as nn
from torchvision.utils import save_image
from tqdm.notebook import tqdm
import torch.nn.functional as F

###參數設定

In [ ]:
#專案資料夾名稱
filenames ="anime_dataset"

#資料庫網址
dataset_url = 'https://www.kaggle.com/datasets/scribbless/another-anime-face-dataset'

#圖片參數設定
image_size = 64
batch_size = 64
stats = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)

#資料庫資料夾位置
# DATA_DIR = '/content/gdrive/MyDrive/anime_dataset/B'

#生成圖片儲存位置
sample_dir = 'generated15'
os.makedirs(sample_dir, exist_ok=True)

#學習率
lr = 5e-5

#算圖次數
epochs = 900

latent_size = 128

g_num =64

d_num = 5

LAMBDA_GP = 10

Clip = 0.01

gp = False

###連google

In [ ]:
drive.mount("/content/gdrive")
path = "/content/gdrive/MyDrive"
os.chdir(path) #切換到要存取資料夾

#創建資料夾
if not os.path.isdir(filenames):
  os.makedirs(filenames)
path2 = f"/content/gdrive/MyDrive/{filenames}"
os.chdir(path2) #切換到要存取資料夾

Mounted at /content/gdrive


###各種函式

In [ ]:
#合成照片
def denorm(img_tensors):
    return img_tensors * stats[1][0] + stats[0][0]

def show_images(images, nmax=64):
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.set_xticks([])
    ax.set_yticks([])

    #make_grid用於合成圖像
    ax.imshow(make_grid(denorm(images.detach()[:nmax]), nrow=8).permute(1, 2, 0))


def show_batch(dl, nmax=64):
    for images, _ in dl:
        show_images(images, nmax)
        break

#用GPU

def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

###下載資料庫&載入資料庫

In [ ]:
DATA_DIR = '/content/gdrive/MyDrive/anime_dataset/A'

train_ds = ImageFolder(DATA_DIR, transform=T.Compose([
    T.Resize(image_size),
    # T.CenterCrop(image_size),
    T.ToTensor(),
    T.Normalize(*stats)]))

train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=3, pin_memory=True)

# for real_images, _ in tqdm(train_dl):
#     real_images = real_images.cpu()
#     print(real_images.shape)
#     for IM in real_images:
#       show_images(IM)
#     break

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
# od.download(dataset_url)

###判別器神經網路

In [ ]:
discriminator = nn.Sequential(
    # in: 3 x 64 x 64

    nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(64),
    nn.LeakyReLU(0.2, inplace=True),
    # out: 64 x 32 x 32

    nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(128),
    nn.LeakyReLU(0.2, inplace=True),
    # out: 128 x 16 x 16

    nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(256),
    nn.LeakyReLU(0.2, inplace=True),
    # out: 256 x 8 x 8

    nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(512),
    nn.LeakyReLU(0.2, inplace=True),
    # out: 512 x 4 x 4

    nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=0, bias=False),
    # out: 1 x 1 x 1

    # nn.Flatten(),
    # nn.Sigmoid()
)

###顯卡用
discriminator = discriminator.to('cuda')

###生成器神經網路

In [ ]:
generator  = nn.Sequential(
    # in: latent_size x 1 x 1

    nn.ConvTranspose2d(latent_size, 512, kernel_size=4, stride=1, padding=0, bias=False),
    nn.BatchNorm2d(512),
    nn.ReLU(True),
    # out: 512 x 4 x 4

    nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(256),
    nn.ReLU(True),
    # out: 256 x 8 x 8

    nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(128),
    nn.ReLU(True),
    # out: 128 x 16 x 16

    nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(64),
    nn.ReLU(True),
    # out: 64 x 32 x 32

    nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1, bias=False),
    nn.Tanh()
    # out: 3 x 64 x 64
)

###顯卡用
generator  = generator .to('cuda')

###顯卡裝置

In [ ]:
#顯卡是否能用
if torch.cuda.is_available():
    # 可以使用 CUDA
    print('GPU OK')
else:
    # 無法使用 CUDA
    print('haha, please use CPU')

GPU OK


In [ ]:
device = get_default_device()
train_dl = DeviceDataLoader(train_dl, device)
generator = to_device(generator, device)

###算圖

In [ ]:
def gradient_penalty(critic, real, fake, device="cpu"):
    BATCH_SIZE, C, H, W = real.shape
    alpha = torch.rand((BATCH_SIZE, 1, 1, 1)).repeat(1, C, H, W).to(device)
    try:
      interpolated_images = real * alpha + fake * (1 - alpha)

      # Calculate critic scores
      mixed_scores = critic(interpolated_images)

      # Take the gradient of the scores with respect to the images
      gradient = torch.autograd.grad(
          inputs=interpolated_images,
          outputs=mixed_scores,
          grad_outputs=torch.ones_like(mixed_scores),
          create_graph=True,
          retain_graph=True,
      )[0]
      gradient = gradient.view(gradient.shape[0], -1)
      gradient_norm = gradient.norm(2, dim=1)
      gradient_penalty = torch.mean((gradient_norm - 1) ** 2)
      return gradient_penalty
    except:
      return 0

def w_distance(fake, rimg=None, fimg=None, real=None, gp = gp):

  if real is None:
      return -torch.mean(fake)
  if gp is True:
      gp = gradient_penalty(discriminator, rimg, fimg, device='cuda')
      return torch.mean(fake) - torch.mean(real)+ LAMBDA_GP * gp
  else:
    return torch.mean(fake) - torch.mean(real)

def initialize_weights(model):
    # Initializes weights according to the DCGAN paper
    for m in model.modules():
        if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
            nn.init.normal_(m.weight.data, 0.0, 0.02)

In [ ]:
#定義判別器和生成器的判斷標準和誤差反向傳遞
generator = to_device(generator, device)

def train_discriminator(real_images, clip, opt_d, gp = gp):
    opt_d.zero_grad()

    latent = torch.randn(batch_size, latent_size, 1, 1, device=device)
    fake_images = generator(latent)

    real_preds = discriminator(real_images).reshape(-1)
    fake_preds = discriminator(fake_images).reshape(-1)

    fake_score = torch.mean(fake_preds).item()
    real_score = torch.mean(real_preds).item()

    loss = w_distance(fake_preds, rimg= real_images, fimg= fake_images, real = real_preds)
    discriminator.zero_grad()
    loss.backward(retain_graph=True)
    opt_d.step()

    if gp == False:
      for module in discriminator.parameters():
        module.data.clamp_(-clip, clip)

    return loss.item(), real_score, fake_score

def train_generator(opt_g):
    opt_g.zero_grad()

    latent = torch.randn(batch_size, latent_size, 1, 1, device=device)
    fake_images = generator(latent)

    preds = discriminator(fake_images)

    loss = w_distance(preds, rimg=None, fimg=None, real=None)

    loss.backward()
    opt_g.step()

    return loss.item()

#儲存算圖結果
def save_samples(index, latent_tensors, epoch, show=True):
    a = 0
    m =  f'epoch:{epoch}/'
    if not os.path.isdir(sample_dir+ '/' +m):
      os.mkdir(sample_dir+ '/' +m)
    fake_images = generator(latent_tensors)
    fake_fname = m +'generated-images-{0:0=4d}'.format(index)
    for a in range(10):
      save_image(denorm(fake_images[a]), os.path.join(sample_dir, fake_fname + str(a)+ '.png'))
      print('Saving', fake_fname)
    if show:
        fig, ax = plt.subplots(figsize=(8, 8))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(fake_images.cpu().detach(), nrow=8).permute(1, 2, 0))

fixed_latent = torch.randn(64, latent_size, 1, 1, device=device)

#算圖主程序
def fit(epochs, lr, clip, start_idx=1):
    torch.cuda.empty_cache()

    losses_g = []
    losses_d = []
    real_scores = []
    fake_scores = []

    opt_d = torch.optim.RMSprop(discriminator.parameters(), lr=lr)
    opt_g = torch.optim.RMSprop(generator.parameters(), lr=lr)

    for epoch in range(epochs):
        for real_images, _ in tqdm(train_dl):
          for _ in range(d_num):
            real_images = real_images.to('cuda')
            loss_d, real_score, fake_score = train_discriminator(real_images, clip, opt_d)

          loss_g = train_generator(opt_g)

        losses_g.append(loss_g)
        losses_d.append(loss_d)
        real_scores.append(real_score)
        fake_scores.append(fake_score)

        print("Epoch [{}/{}], loss_g: {:.4f}, loss_d: {:.4f}, real_score: {:.4f}, fake_score: {:.4f}".format(
            epoch+1, epochs, loss_g, loss_d, real_score, fake_score))

         save_samples(epoch+start_idx, fixed_latent, epoch, show=False)

        # # Save the model checkpoints
        torch.save(generator.state_dict(), f'{path2}/{sample_dir}/G.pth')
        torch.save(discriminator.state_dict(), f'{path2}/{sample_dir}/D.pth')

    return losses_g, losses_d, real_scores, fake_scores

initialize_weights(generator)
initialize_weights(discriminator)
losses_g, losses_d, real_scores, fake_scores = fit(epochs, lr, Clip)

  0%|          | 0/222 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch [1/900], loss_g: 0.6792, loss_d: -1.4925, real_score: 0.8134, fake_score: -0.6791
Saving epoch:0/generated-images-0001
Saving epoch:0/generated-images-0001
Saving epoch:0/generated-images-0001
Saving epoch:0/generated-images-0001
Saving epoch:0/generated-images-0001
Saving epoch:0/generated-images-0001
Saving epoch:0/generated-images-0001
Saving epoch:0/generated-images-0001
Saving epoch:0/generated-images-0001
Saving epoch:0/generated-images-0001


  0%|          | 0/222 [00:00<?, ?it/s]

Epoch [2/900], loss_g: 0.6814, loss_d: -1.4980, real_score: 0.8166, fake_score: -0.6814
Saving epoch:1/generated-images-0002
Saving epoch:1/generated-images-0002
Saving epoch:1/generated-images-0002
Saving epoch:1/generated-images-0002
Saving epoch:1/generated-images-0002
Saving epoch:1/generated-images-0002
Saving epoch:1/generated-images-0002
Saving epoch:1/generated-images-0002
Saving epoch:1/generated-images-0002
Saving epoch:1/generated-images-0002


...

  0%|          | 0/222 [00:00<?, ?it/s]

Epoch [130/900], loss_g: 0.1597, loss_d: -0.7050, real_score: 0.2283, fake_score: -0.4767
Saving epoch:129/generated-images-0130
Saving epoch:129/generated-images-0130
Saving epoch:129/generated-images-0130
Saving epoch:129/generated-images-0130
Saving epoch:129/generated-images-0130
Saving epoch:129/generated-images-0130
Saving epoch:129/generated-images-0130
Saving epoch:129/generated-images-0130
Saving epoch:129/generated-images-0130
Saving epoch:129/generated-images-0130


  0%|          | 0/222 [00:00<?, ?it/s]

Epoch [131/900], loss_g: 0.1482, loss_d: -0.6937, real_score: 0.2303, fake_score: -0.4633
Saving epoch:130/generated-images-0131
Saving epoch:130/generated-images-0131
Saving epoch:130/generated-images-0131
Saving epoch:130/generated-images-0131
Saving epoch:130/generated-images-0131
Saving epoch:130/generated-images-0131
Saving epoch:130/generated-images-0131
Saving epoch:130/generated-images-0131
Saving epoch:130/generated-images-0131
Saving epoch:130/generated-images-0131


  0%|          | 0/222 [00:00<?, ?it/s]

###顯示最終算圖圖片

In [ ]:
Image(f'./generated/generated-images-00{str(epochs)}.png')